# **Base model with embedding watermark**


Epoch [1/10], Test Accuracy: 64.96%, Classification Loss: 1.7705, Watermark Loss: 0.0070

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [1. 1. 0. 0. 0. 1. 1. 0.]

BER 0.75
Epoch [2/10], Test Accuracy: 97.11%, Classification Loss: 1.5053, Watermark Loss: 0.0069

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 0. 0. 0. 1. 0.]

BER 0.5
Epoch [3/10], Test Accuracy: 97.67%, Classification Loss: 1.4900, Watermark Loss: 0.0068

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 0. 1. 0. 1. 0.]

BER 0.375
Epoch [4/10], Test Accuracy: 98.05%, Classification Loss: 1.4667, Watermark Loss: 0.0068

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 0. 1. 0. 1. 0.]

BER 0.375
Epoch [5/10], Test Accuracy: 98.00%, Classification Loss: 1.4613, Watermark Loss: 0.0067

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 0. 1. 0. 1. 0.]

BER 0.375
Epoch [6/10], Test Accuracy: 98.17%, Classification Loss: 1.4720, Watermark Loss: 0.0066

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 1. 1. 0. 1. 0.]

BER 0.25
Epoch [7/10], Test Accuracy: 98.36%, Classification Loss: 1.4615, Watermark Loss: 0.0066

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

BER 0.0
✅ Model saved with Test Accuracy: 98.36% and BER: 0.0000 and bestepoch: 7
Epoch [8/10], Test Accuracy: 98.38%, Classification Loss: 1.4703, Watermark Loss: 0.0065

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

BER 0.0
✅ Model saved with Test Accuracy: 98.38% and BER: 0.0000 and bestepoch: 8
Epoch [9/10], Test Accuracy: 98.46%, Classification Loss: 1.4671, Watermark Loss: 0.0064

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

BER 0.0
✅ Model saved with Test Accuracy: 98.46% and BER: 0.0000 and bestepoch: 9
Epoch [10/10], Test Accuracy: 98.42%, Classification Loss: 1.4612, Watermark Loss: 0.0064

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

BER 0.0
✅Final  Model Test Accuracy: 98.46% and BER: 0.0000 and bestepoch: 9



In [6]:
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import random

import torch.optim.lr_scheduler as lr_scheduler

In [7]:
# Set random seed for reproducibility
seed = 58  # You can choose any number
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If using multiple GPUs
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
torch.backends.cudnn.benchmark = False  # Disable benchmark for reproducibility


In [8]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [10]:

# Hyperparameters
num_epochs =10
batch_size = 256
learning_rate =0.001
lambda_wm =1e-2 # 1e-2 Watermark regularizer 0.01
best_acc = 0.0  # Track best test accuracy
best_ber = float("inf")  # Track lowest BER
best_model_path = "best_model.pth"


In [11]:
# Load dataset


# !rm -rf ./data/MNIST



transform = transforms.Compose([
    # transforms.RandomRotation(10),  # Rotate images by ±10 degrees
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])


train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)  # Test set

# Use the same seed for the DataLoader shuffle
g = torch.Generator()
g.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=lambda _: np.random.seed(seed), generator=g)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [12]:
# Computes the Bit Error Rate (BER)
def compute_ber(original_watermark, extracted_watermark):
  diff = original_watermark - extracted_watermark
  num_errors = torch.sum(torch.abs(diff))
  ber = num_errors.float() / original_watermark.numel()
  return ber.item() #float


In [14]:
# Define Watermark Regularizer
class WatermarkRegularizer(nn.Module):
    def __init__(self, lambda_wm, watermark_vector, C_in, K):
        super(WatermarkRegularizer, self).__init__()
        self.lambda_wm = lambda_wm  # Watermark regularizer
        self.watermark_vector = watermark_vector  #  watermark vector
        T=watermark_vector.shape[0]
        M = C_in*K*K  # Hidden dimension for projection
        self.secret_key = torch.randn(T, M, device=device)
        # self.secret_key = torch.nn.functional.normalize(self.secret_key, p=2, dim=1)
        # print(self.secret_key.shape)   #8,9

    def forward(self, weights):
        # print(weights.size())   #32,1,3,3
        w_mean = weights.mean(dim=(0))  # mean of filters
        # print(w_mean.size())   #1*3*3
        w_mean_flat = w_mean.view(-1)  # Flatten(C_in * K * K)
        # print(w_mean_flat.size())   #9
        # print(self.secret_key.T.size())  #9,8
        projected_wm = torch.sigmoid(torch.matmul (self.secret_key,w_mean_flat))  # Compute WX
        # wm_loss=self.lambda_wm * torch.norm(projected_wm - self.watermark_vector)  # Regularization loss
        wm_loss = self.lambda_wm * nn.BCELoss(reduction='mean')(projected_wm.to(device), self.watermark_vector.to(device))
        # print((projected_wm > 0.5).float())
        # print(self.watermark_vector)
        # print('***************************************')

        return wm_loss

In [15]:
# Define a Simple CNN with Watermark Embedding
class WatermarkedCNN(nn.Module):
    def __init__(self):
        super(WatermarkedCNN, self).__init__()
        # self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 28 *28, 512)
        self.fc2 = nn.Linear(512, 10)
        # self.wm_regularizer = WatermarkRegularizer(lambda_wm, watermark_vector, C_in=1, K=3)

    def forward(self, x):
        x = torch.relu(self.conv2(x)) # watermark is here
        # print(x.shape)
        # wm_loss = self.wm_regularizer(self.conv2.weight)
        x = x.view(x.size(0), -1)
        # print(x.shape)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        # x=torch.sigmoid(x)
        x = torch.softmax(x, dim=1)
        return x#, wm_loss


In [ ]:

# generate a random watermark, ignoring the seed
# rand_gen = torch.Generator(device)  # Create a new generator
# rand_gen.seed()  # Seed it randomly

# # Generate a new random watermark vector using this independent generator
# watermark_vector = torch.randint(0, 2, (256,), dtype=torch.float32, device=device, generator=rand_gen)


# watermark_vector = torch.randint(0, 2, (256,), dtype=torch.float32, device=device)  # Binary watermark



watermark_vector=torch.tensor([0., 1., 0., 1., 1., 0., 0., 1.], dtype=torch.float32)

# print(watermark_vector)
# print(watermark_vector.size()) #8

In [12]:

#BASE_EMBEDDED_MODEL

model = WatermarkedCNN().to(device)
wm_regularizer = WatermarkRegularizer(lambda_wm, watermark_vector, C_in=1, K=3)
torch.save(wm_regularizer.secret_key, "secret_key.pth")  # Save the secret key
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss_class = criterion(outputs, labels)
        wm_loss = wm_regularizer(model.conv2.weight)
        loss = loss_class + wm_loss  # Total loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # current_lr = optimizer.param_groups[0]['lr']
    # print(f"Epoch {epoch+1}/{num_epochs}, Learning Rate: {current_lr:.6f}")
    # scheduler.step()

    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No gradients needed
          for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get class index with highest probability
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # Count correct predictions
    accuracy = 100 * correct / total  # Compute accuracy percentage

    print(f"Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%, Classification Loss: {loss_class.item():.4f}, "
          f"Watermark Loss: {wm_loss.item():.4f}")
    # print(f"Epoch [{epoch+1}/{num_epochs}],  Classification Loss: {loss_class.item():.4f}, "
    #       f"Watermark Loss: {wm_loss.item():.4f}")

    with torch.no_grad():
      conv2_mean = model.conv2.weight.mean(dim=0)  # Should be (1,3,3)
      conv2_mean_flat = conv2_mean.view(-1)
      extracted_watermark = torch.sigmoid(torch.matmul( wm_regularizer.secret_key.cpu(), conv2_mean_flat.cpu()))  # Fix dimensions
      extracted_watermark_binary = (extracted_watermark > 0.5).float()
      # print(type(extracted_watermark_binary))
      # print(type(watermark_vector))
      print("\nOriginal Watermark:", watermark_vector.cpu().numpy())
      print("\nExtracted Watermark:", extracted_watermark_binary.cpu().numpy())
      print("\nBER", compute_ber(watermark_vector.cpu(), extracted_watermark_binary.cpu()))

    ber=compute_ber(watermark_vector.cpu(), extracted_watermark_binary.cpu())
    if accuracy>best_acc and ber<=0.0:
        if os.path.exists(best_model_path):
          os.remove(best_model_path)
        best_acc=accuracy
        best_ber=ber
        best_epoch=epoch+1
        best_model_path = f"best_model_lR:{learning_rate}_lamda:{lambda_wm}_Acc:{best_acc}_Epoch:{best_epoch}_Ber:{best_ber}.pth"
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Model saved with Test Accuracy: {best_acc:.2f}% and BER: {best_ber:.4f} and bestepoch: {best_epoch}")

print(f"✅Final  Model Test Accuracy: {best_acc:.2f}% and BER: {best_ber:.4f} and bestepoch: {best_epoch}")








Epoch [1/10], Test Accuracy: 86.15%, Classification Loss: 1.5913, Watermark Loss: 0.0068

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 1. 1. 1. 1. 1. 0.]

BER 0.5
Epoch [2/10], Test Accuracy: 87.35%, Classification Loss: 1.6720, Watermark Loss: 0.0066

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 1. 1. 1. 1. 0. 1.]

BER 0.25
Epoch [3/10], Test Accuracy: 88.01%, Classification Loss: 1.5680, Watermark Loss: 0.0065

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

BER 0.0
✅ Model saved with Test Accuracy: 88.01% and BER: 0.0000 and bestepoch: 3
Epoch [4/10], Test Accuracy: 97.98%, Classification Loss: 1.4762, Watermark Loss: 0.0064

Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

BER 0.0
✅ Model saved with Test Accuracy: 97.98% and BER: 0.0000 and bestepoch: 4
Epoch [5/10], Test Accuracy: 98.09%, Classification Loss: 1.4861, Watermark Los

# **Base model without watermark**

In [22]:
#BASE_MODEL_NOT_EMBEDED
# # generate a random watermark, ignoring the seed
# rand_gen = torch.Generator(device)  # Create a new generator
# rand_gen.seed()  # Seed it randomly
best_model_path = "best_model2.pth"


model = WatermarkedCNN().to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss_class = criterion(outputs, labels)
        loss = loss_class  # Total loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No gradients needed
          for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get class index with highest probability
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = 100 * correct / total  # Compute accuracy percentage

    print(f"Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%, Classification Loss: {loss_class.item():.4f}")



    if accuracy>best_acc :
        if os.path.exists(best_model_path):
          os.remove(best_model_path)
        best_acc=accuracy
        best_epoch=epoch+1
        best_model_path = f"best_model_lR:{learning_rate}_Acc:{best_acc}_Epoch:{best_epoch}.pth"
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Model saved with Test Accuracy: {best_acc:.2f}%  and bestepoch: {best_epoch}")


print(f"✅Final  Model Test Accuracy: {best_acc:.2f}% and bestepoch: {best_epoch}")



Epoch [1/10], Test Accuracy: 86.19%, Classification Loss: 1.5423
Epoch [2/10], Test Accuracy: 87.02%, Classification Loss: 1.5721
Epoch [3/10], Test Accuracy: 88.05%, Classification Loss: 1.5643
Epoch [4/10], Test Accuracy: 97.81%, Classification Loss: 1.4625
Epoch [5/10], Test Accuracy: 97.85%, Classification Loss: 1.4852
Epoch [6/10], Test Accuracy: 98.49%, Classification Loss: 1.4727
✅ Model saved with Test Accuracy: 98.49%  and bestepoch: 6
Epoch [7/10], Test Accuracy: 98.46%, Classification Loss: 1.4615
Epoch [8/10], Test Accuracy: 98.21%, Classification Loss: 1.4623
Epoch [9/10], Test Accuracy: 98.52%, Classification Loss: 1.4716
✅ Model saved with Test Accuracy: 98.52%  and bestepoch: 9
Epoch [10/10], Test Accuracy: 98.19%, Classification Loss: 1.4877
✅Final  Model Test Accuracy: 98.52% and bestepoch: 9


# **base:embeded model fine tuning without embeding- fine tune all parametres**



Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

BER 0.0

In [76]:
#fine tunning with the dataSET to see watermark still exist or not
rand_gen = torch.Generator(device)  # Create a new generator
rand_gen.seed()  # Seed it randomly
# Assuming you have saved your model as 'model.pth'
LR=0.0001
model_finetune = WatermarkedCNN().to(device)  # Assuming WatermarkedCNN is your model class
model_finetune.load_state_dict(torch.load('BASE-EMBEDDED_MODEL_lR:0.001_lamda:0.01_Acc:97.98_Epoch:4_Ber:0.0.pth'))
# model_finetune.fc2 = nn.Linear(512, 10).to(device)
model_finetune.train()


# for name, param in model_finetune.named_parameters():
#     print(f"{name}: requires_grad={param.requires_grad}")

optimizer = optim.Adam(model_finetune.parameters(), lr=LR)  #fine tune all parameteres
# optimizer = optim.Adam(model_finetune.fc2.parameters(), lr=1e-3) #finetune just last layer






In [77]:

num_finetune_epochs = 10  # Adjust as needed
best_finetune_acc = 0.0  # Track best test accuracy
best_model_path = "best_model_finetune.pth"
for epoch in range(num_finetune_epochs):
    model_finetune.train()  # Ensure model is in training mode

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_finetune(images)

        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()

        # for name, param in model_finetune.named_parameters():
        #   if param.requires_grad and param.grad is not None:
        #       print(f"{name} - Grad Norm: {param.grad.norm().item()}")
        #
        #
        # for name, param in model_finetune.named_parameters():
        #    if param.requires_grad:
        #       print(f"{name} - Grad After Backward: {param.grad}")

        # for param_group in optimizer.param_groups:
        #     print(param_group['lr'])


        optimizer.step()

        # print(f"Epoch {epoch}, Loss: {loss.item()}")



    model_finetune.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No gradients needed
          for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs =model_finetune(images)
            _, predicted = torch.max(outputs, 1)  # Get class index with highest probability
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = 100 * correct / total  # Compute accuracy percentage

    print(f"Epoch [{epoch+1}/{num_finetune_epochs}], Test Accuracy: {accuracy:.2f}%, Classification Loss: {loss.item():.4f}")

    # Save the fine-tuned model
    if accuracy>best_finetune_acc :
        if os.path.exists(best_model_path):
          os.remove(best_model_path)
        best_finetune_acc =accuracy
        best_epoch=epoch+1
        best_model_path = f"best_finetuned_model_lR:{LR}_Acc:{best_finetune_acc}_Epoch:{best_epoch}.pth"
        torch.save(model_finetune.state_dict(), best_model_path)
        print(f"✅ Model saved with Test Accuracy: {best_finetune_acc:.2f}% and bestepoch: {best_epoch}")







Epoch [1/10], Test Accuracy: 98.43%, Classification Loss: 1.4705
✅ Model saved with Test Accuracy: 98.43% and bestepoch: 1


KeyboardInterrupt: 

In [62]:
model_finetune2 = WatermarkedCNN().to(device)  # Assuming WatermarkedCNN is your model class

model_finetune2.load_state_dict(torch.load('0best_finetuned_model_lR:0.0001_Acc:98.67_Epoch:8.pth'))


with torch.no_grad():
      conv2_mean = model_finetune2.conv2.weight.mean(dim=0)  # Should be (1,3,3)
      conv2_mean_flat = conv2_mean.view(-1)
      extracted_watermark = torch.sigmoid(torch.matmul( wm_regularizer.secret_key.cpu(), conv2_mean_flat.cpu()))  # Fix dimensions
      extracted_watermark_binary = (extracted_watermark > 0.5).float()

      # print(type(extracted_watermark_binary))
      # print(type(watermark_vector))

      print("\nOriginal Watermark:", watermark_vector.cpu().numpy())
      print("\nExtracted Watermark:", extracted_watermark_binary.cpu().numpy())
      print("\nBER", compute_ber(watermark_vector.cpu(), extracted_watermark_binary.cpu()))

#finish FINE_TUNING

NameError: name 'wm_regularizer' is not defined

#**base:NOT embeded model fine tuning without embeding**

In [239]:
#pruning the weights of the model


rand_gen = torch.Generator(device)  # Create a new generator
rand_gen.seed()  # Seed it randomly

LR=0.0001
model_prun= WatermarkedCNN().to(device)  # Assuming WatermarkedCNN is your model class
model_prun.load_state_dict(torch.load('BASE-EMBEDDED_MODEL_lR:0.001_lamda:0.01_Acc:97.98_Epoch:4_Ber:0.0.pth'))
# model_prun.eval()

optimizer = optim.Adam(model_prun.parameters(), lr=LR)  #fine tune all parameteres
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model_finetune.fc2.parameters(), lr=1e-3) #finetune just last layer


In [240]:
import copy
model_copy=copy.deepcopy(model_prun)

In [191]:
state_dict1 = model_prun.state_dict()
print(state_dict1)

OrderedDict({'conv2.weight': tensor([[[[ 0.0364, -0.0118,  0.0811],
          [-0.1568,  0.3304, -0.0164],
          [-0.1316, -0.2825, -0.0853]]],


        [[[-0.1040, -0.2836,  0.2635],
          [-0.3641,  0.2070,  0.3110],
          [-0.0376,  0.2994, -0.1587]]],


        [[[-0.3878,  0.0688,  0.2889],
          [ 0.3228, -0.0161, -0.0536],
          [ 0.0370,  0.0169, -0.2963]]],


        [[[-0.1488, -0.0051, -0.2002],
          [ 0.3119, -0.1132, -0.3218],
          [-0.1379,  0.3127,  0.3454]]],


        [[[-0.0908, -0.1422,  0.3827],
          [-0.0132, -0.2228, -0.0827],
          [-0.0620, -0.2853, -0.2536]]],


        [[[ 0.1208,  0.2487, -0.1315],
          [-0.0936,  0.3515, -0.1770],
          [-0.3719, -0.2072,  0.2853]]],


        [[[ 0.0119,  0.2366, -0.4234],
          [ 0.3459,  0.1551, -0.3175],
          [-0.2677,  0.3744,  0.0044]]],


        [[[-0.3824, -0.0374,  0.3677],
          [-0.4045, -0.0400,  0.3591],
          [-0.3629,  0.1721,  0.3986]]],


   

In [241]:
secret_key = torch.load("secret_key.pth")
watermark_vector=torch.tensor([0., 1., 0., 1., 1., 0., 0., 1.], dtype=torch.float32)

with torch.no_grad():
      conv2_mean = model_prun.conv2.weight.mean(dim=0)  # Should be (1,3,3)
      conv2_mean_flat = conv2_mean.view(-1)
      extracted_watermark = torch.sigmoid(torch.matmul( secret_key .cpu(), conv2_mean_flat.cpu()))  # Fix dimensions
      extracted_watermark_binary = (extracted_watermark > 0.5).float()

      # print(type(extracted_watermark_binary))
      # print(type(watermark_vector))

      print("\nOriginal Watermark:", watermark_vector.cpu().numpy())
      print("\nExtracted Watermark:", extracted_watermark_binary.cpu().numpy())
      print("\nBER", compute_ber(watermark_vector.cpu(), extracted_watermark_binary.cpu()))


Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

BER 0.0


In [242]:


def prune_smallest_percentage(model, percentage=0.1):
    # Loop through model layers and prune based on smallest weights
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            weight_tensor = module.weight.data

            # Get the absolute value of weights and sort them
            abs_weights = torch.abs(weight_tensor).view(-1)  # Flatten to 1D
            sorted_weights, indices = torch.sort(abs_weights)  # Sort by absolute values
            print('sorted_weights')
            print(sorted_weights)

            # Calculate the threshold for the smallest 'percentage' of weights
            print(torch.max(sorted_weights))
            num_weights_to_prune = int((len(sorted_weights) * percentage))
            print('num_weights_to_prune')
            print(num_weights_to_prune)
            if num_weights_to_prune == len(sorted_weights):
                threshold = sorted_weights[num_weights_to_prune-1]
            else:
                threshold = sorted_weights[num_weights_to_prune]
            print(threshold)
            print('threshold')

            # Create a pruning mask: set weights smaller than the threshold to zero
            mask = torch.abs(weight_tensor) < threshold

            # Apply the mask to prune weights
            weight_tensor1=weight_tensor
            print(weight_tensor)
            print('weight_tensor1')
            weight_tensor[mask] = 0
            print(weight_tensor)
            print('weight_tensor')

            print("\nBER", compute_ber(weight_tensor1.cpu(), weight_tensor.cpu()))


            # Optionally, you can apply pruning to the layer (keeping track of mask)
            # prune.custom_from_mask(module, 'weight', mask)
            print(f"Pruned {num_weights_to_prune} weights in layer {name}")

    return model


model_prun1= prune_smallest_percentage(model_copy, percentage=1.0)


sorted_weights
tensor([0.0011, 0.0035, 0.0038, 0.0042, 0.0044, 0.0051, 0.0062, 0.0076, 0.0092,
        0.0118, 0.0119, 0.0123, 0.0123, 0.0132, 0.0139, 0.0146, 0.0161, 0.0164,
        0.0169, 0.0180, 0.0188, 0.0189, 0.0214, 0.0231, 0.0232, 0.0247, 0.0260,
        0.0271, 0.0290, 0.0304, 0.0310, 0.0316, 0.0322, 0.0346, 0.0348, 0.0356,
        0.0364, 0.0370, 0.0374, 0.0376, 0.0399, 0.0400, 0.0416, 0.0466, 0.0484,
        0.0522, 0.0532, 0.0536, 0.0565, 0.0582, 0.0586, 0.0620, 0.0632, 0.0636,
        0.0679, 0.0680, 0.0681, 0.0688, 0.0698, 0.0707, 0.0722, 0.0730, 0.0744,
        0.0772, 0.0776, 0.0783, 0.0793, 0.0811, 0.0820, 0.0827, 0.0844, 0.0853,
        0.0857, 0.0879, 0.0889, 0.0892, 0.0900, 0.0903, 0.0904, 0.0905, 0.0908,
        0.0931, 0.0935, 0.0936, 0.0949, 0.0993, 0.1014, 0.1015, 0.1040, 0.1041,
        0.1093, 0.1123, 0.1130, 0.1132, 0.1143, 0.1146, 0.1208, 0.1241, 0.1258,
        0.1270, 0.1282, 0.1299, 0.1315, 0.1316, 0.1318, 0.1323, 0.1350, 0.1351,
        0.1354, 0.1362, 0

# **base embeded--- model fine tune with same embeding**

In [243]:
state_dict1 = model_prun.state_dict()
state_dict2 = model_prun1.state_dict()

for key in state_dict1:
    if torch.equal(state_dict1[key], state_dict2[key]):
        # print(state_dict1[key])
        # print('*******************')
        # print(state_dict2[key])
        print(f"Layer {key}: Weights are identical.")
    else:
        print(f"Layer {key}: Weights are different.")


Layer conv2.weight: Weights are different.
Layer conv2.bias: Weights are identical.
Layer fc1.weight: Weights are different.
Layer fc1.bias: Weights are identical.
Layer fc2.weight: Weights are different.
Layer fc2.bias: Weights are identical.


In [244]:
secret_key = torch.load("secret_key.pth")
watermark_vector=torch.tensor([0., 1., 0., 1., 1., 0., 0., 1.], dtype=torch.float32)

with torch.no_grad():
      conv2_mean = model_prun1.conv2.weight.mean(dim=0)  # Should be (1,3,3)
      conv2_mean_flat = conv2_mean.view(-1)
      extracted_watermark = torch.sigmoid(torch.matmul( secret_key .cpu(), conv2_mean_flat.cpu()))  # Fix dimensions
      extracted_watermark_binary = (extracted_watermark > 0.5).float()

      # print(type(extracted_watermark_binary))
      # print(type(watermark_vector))

      print("\nOriginal Watermark:", watermark_vector.cpu().numpy())
      print("\nExtracted Watermark:", extracted_watermark_binary.cpu().numpy())
      print("\nBER", compute_ber(watermark_vector.cpu(), extracted_watermark_binary.cpu()))

#finish PRUNING


Original Watermark: [0. 1. 0. 1. 1. 0. 0. 1.]

Extracted Watermark: [0. 1. 0. 0. 0. 1. 0. 1.]

BER 0.375
